In [1]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Example generate text function
def generate_text(model, device, char_idx_map, idx_to_char, max_len=1000, temp=0.8):
    model.eval()
    start_text = "Macbeth\n by William Shakespeare\n Edited by Barbara A. Mowat and Paul Werstine"
    input_seq = [char_idx_map[c] for c in start_text]
    input_tensor = torch.tensor(input_seq, dtype=torch.long).unsqueeze(0).to(device)
    
    model.to(device)
    generated_text = start_text
    for _ in range(max_len):
        with torch.no_grad():
            output, hidden = model(input_tensor)
            output = output.squeeze().div(temp).exp()
            predicted_idx = torch.multinomial(output, 1).item()
            predicted_char = idx_to_char[predicted_idx]
            generated_text += predicted_char
            input_tensor = torch.cat((input_tensor[:, 1:], torch.tensor([[predicted_idx]], dtype=torch.long).to(device)), dim=1)
    
    return generated_text

In [ ]:
# Example of function being called
input_file = 'tiny_shakespeare.txt'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

encoded_text, vocab_size, char_to_idx, idx_to_char = encode_text(input_file)

### Load the right config
config_path = 'base_lstm_config.yaml' #'lstm_512_config.yaml' 
config = load_config(config_path)


#### Load your trained model
model = LSTMModel(vocab_size=vocab_size,
                 embed_size=config['embed_size'],
                hidden_size=config['hidden_size'],
                num_layers=config['num_layers'])

model_path = 'best_model_128_deep.pth' #'best_model.pth' #'LSTM_seq_len_4.pth'
model.load_state_dict(torch.load(model_path, weights_only=True))

# Example usage
generated = generate_text(model, device, char_to_idx, idx_to_char, max_len=1000, temp=1)
print(generated)